<a href="https://colab.research.google.com/github/Atulkhiyani0909/ByteVerse_NayaSetu/blob/main/Chatbot_RAG/NyayaSetu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>